## Management and Analysis of Physics Dataset - mod.B

## Final project: Streaming processing of cosmic rays using Drift Tubes detectors

The goal of this project is to reproduce a real-time processing of real data collected in a particle physics detector and publish the results in a dashboard for live monitoring.

### Students:
* Conforto Filippo (2021856)
* Domenichetti Lorenzo (2011653)
* Faorlin Tommaso (2021857)

## Structured Streaming notebook

In [1]:
import json
import time

import findspark
import numpy as np
from numpy import arange
from numpy import linspace 

from kafka import KafkaProducer
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.streaming import StreamingContext
from pyspark.sql.types import StructField, StructType, StringType, DoubleType, IntegerType
from pyspark.sql.functions import from_json, col, max, min


## Creating Spark context

In [2]:
findspark.init('/home/packages/spark-3.1.2-bin-hadoop3.2')

In [3]:
spark = SparkSession.builder\
    .master("spark://10.67.22.100:7077")\
    .appName("MAPD Final Project session")\
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
    .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "false")\
    .config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2")\
    .getOrCreate()

In [4]:
spark

## Kafka

In [5]:
KAFKA_BOOTSTRAP_SERVERS = "10.67.22.226:9092" #:226 corresponds to slave 04. 

inputDF = spark\
    .readStream\
    .format("kafka")\
    .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS)\
    .option('subscribe', 'topic_stream')\
    .option("startingOffsets", "latest") \
    .load()

In [6]:
schema = StructType(
        [StructField("HEAD",        IntegerType()),
         StructField("FPGA",         IntegerType()),
         StructField("TDC_CHANNEL",  IntegerType()),
         StructField("ORBIT_CNT",    DoubleType()), ## Double to overcome overflow problems
         StructField("BX_COUNTER",   DoubleType()), ## Double to overcome overflow problems
         StructField("TDC_MEAS",    DoubleType() )]
)

In [7]:
jsonDF = inputDF.select(from_json(col("value").alias('value').cast("string"), schema).alias('value'))

In [8]:
#flatten out the dataframe
flatDF = jsonDF.selectExpr("value.HEAD", 
                           "value.FPGA", 
                           "value.TDC_CHANNEL",
                           "value.ORBIT_CNT",
                           "value.BX_COUNTER",
                           "value.TDC_MEAS")

##  Cleaning the upcoming dataframe

In [9]:
df = flatDF.where(col("HEAD")==2)

## Uploading to consumer

In [10]:
producer = KafkaProducer(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS)

In [11]:
# scintillator time offset by Chamber
time_offset_by_chamber = {
0: 95.0 - 1.1, # Ch 0
1: 95.0 + 6.4, # Ch 1
2: 95.0 + 0.5, # Ch 2
3: 95.0 - 2.6, # Ch 3
}

In [12]:
def batch_proc(batch_df, epoch_id):
    
    hits = batch_df.count()
    
    if hits!=0: 
        start = time.time()
        #Creating the ABSOLUTETIME column
        batch_df = batch_df.withColumn("ABSOLUTETIME", 25*(col('ORBIT_CNT')*1e-9*3564+col('BX_COUNTER')*1e-9+(col('TDC_MEAS')*1e-9)/30))
        
        #Dataframe containing only informations about scintillator events
        batch_df_scint = (batch_df.filter('(FPGA==1) AND (TDC_CHANNEL == 128)')
                        .select(['ORBIT_CNT', 'ABSOLUTETIME'])
                        .groupBy('ORBIT_CNT')
                        .min()
                        .withColumnRenamed("min(ABSOLUTETIME)", 'ABSOLUTETIME_SCINT')
                        .drop('min(ORBIT_CNT)')
                       )
        #Dividing the dataframe between chambers
        
        batch_df_ch0 = batch_df.filter('(FPGA==0) AND (TDC_CHANNEL >= 0) AND (TDC_CHANNEL < 64)')
        batch_df_ch1 = batch_df.filter('(FPGA==0) AND (TDC_CHANNEL >= 64) AND (TDC_CHANNEL < 128)')
        batch_df_ch2 = batch_df.filter('(FPGA==1) AND (TDC_CHANNEL >= 0) AND (TDC_CHANNEL < 64)')
        batch_df_ch3 = batch_df.filter('(FPGA==1) AND (TDC_CHANNEL >= 64) AND (TDC_CHANNEL < 128)')
        
        batch_dfs = [batch_df_ch0, batch_df_ch1, batch_df_ch2, batch_df_ch3]
        
        # Counting hits for each chamber

        hits_ch0 = batch_df_ch0.count()
        hits_ch1 = batch_df_ch1.count()
        hits_ch2 = batch_df_ch2.count()
        hits_ch3 = batch_df_ch3.count()

        end = time.time()-start
        #Total active channels histogram
        
        hist1 = {}
        
        #Channels per orbit histogram
            
        hist2 = {}
        min_hist, max_hist = batch_df.agg(min("ORBIT_CNT"), max("ORBIT_CNT")).rdd.flatMap(lambda x: x).collect()
        binning = list(linspace(0, 4e8, 30))

        #Active channels in orbits in which the scintillator is active histogram
        hist3 = {}

        #Drifttime histogram
        hist4 = {}
        #Binning for driftime - arbitrario perché a volte non troviamo né max né min.. da rivedere.
        binning_drift = list(linspace(0,800, 40))
        
        
        
        for chamber in [0,1,2,3]:
            hist1[chamber] = {}
            hist2[chamber] = {}
            hist3[chamber] = {}
            hist4[chamber] = {}

            bins, counts = (
                batch_dfs[chamber].select('TDC_CHANNEL')
                .rdd.map(lambda x: x.TDC_CHANNEL)
                .histogram(list(arange((chamber % 2)*64,(chamber % 2 +1)*64,1)))
            )
            
            bins2, counts2 = (
                batch_dfs[chamber].groupBy("ORBIT_CNT","TDC_CHANNEL").count()
                .select('ORBIT_CNT')
                .rdd.map(lambda x: x.ORBIT_CNT)
                .histogram(binning)
            )
            
            #Filtering only useful hits (avoid scintillators) and use inner join to consider coincident events
            batch_dfs[chamber] = batch_dfs[chamber].filter('NOT ((FPGA==1) AND (TDC_CHANNEL == 128))')\
                                                   .join(batch_df_scint, ["ORBIT_CNT"], "inner")
                
            #Creating driftime and select only positive values
            batch_dfs[chamber] = batch_dfs[chamber].withColumn('DRIFTIME', (col('ABSOLUTETIME')-col('ABSOLUTETIME_SCINT'))*1e9+time_offset_by_chamber[chamber])\
                                .where(col('DRIFTIME')>0) #Is it possible to remove it?
            
            bins3, counts3 = (
                batch_dfs[chamber].select('TDC_CHANNEL')
                .rdd.map(lambda x: x.TDC_CHANNEL)
                .histogram(list(arange((chamber % 2)*64,(chamber % 2 +1)*64,1)))
            )

            bins4, counts4 = (
                batch_dfs[chamber].select('DRIFTIME')
                .rdd.map(lambda x: x.DRIFTIME)
                .histogram(binning_drift)
            )

            hist1[chamber]['bins'] = list(map(int,bins)) #must convert to python integers
            hist1[chamber]['counts'] = list(map(int,counts))
        
            hist2[chamber]['bins'] = list(map(int,bins2))
            hist2[chamber]['counts'] = list(map(int,counts2))
            
            hist3[chamber]['bins'] = list(map(int,bins3))
            hist3[chamber]['counts'] = list(map(int,counts3))
        
            hist4[chamber]['bins'] = list(map(int,bins4))
            hist4[chamber]['counts'] = list(map(int,counts4))

        #Producing the results dictionary
        result = {
            "hits" : hits,
            "hits_per_chamber": [hits_ch0, hits_ch1, hits_ch2, hits_ch3],
            "hist_1": hist1,
            "hist_2": hist2,
            "hist_3": hist3,
            "hist_4": hist4
        }
        
        #Sending the json to the producer
        producer.send('topic_results', json.dumps(result).encode('utf-8'))
#         producer.flush()
        
    else: 
        pass
    


In [13]:
df.writeStream\
    .trigger(processingTime='5 second')\
    .foreachBatch(batch_proc)\
    .start()\
    .awaitTermination()

KeyboardInterrupt: 

## Backup

In [ ]:
# load dataset on dataset/lecture2/dimuon

schema = StructType()                          \
      .add("HEAD",        IntegerType(), True) \
      .add("FPGA",        IntegerType(), True) \
      .add("TDC_CHANNEL", IntegerType(), True) \
      .add("ORBIT_CNT",   IntegerType(), True) \
      .add("BX_COUNTER",  IntegerType(), True) \
      .add("TDC_MEAS",    DoubleType(),  True)

df = spark.read.format("csv") \
      .option("header",True) \
      .schema(schema) \
      .load("/home/data_000019.txt")

df = df.where(col("HEAD")==2)
#df = df.withColumn("ABSOLUTETIME", 25*(col('ORBIT_CNT')*1e-9*3564+col('BX_COUNTER')*1e-9+(col('TDC_MEAS')*1e-9)/30))

In [ ]:
df.rdd.collect()[3]

In [18]:
import time

def batch_proc(batch_df, epoch_id):
    
    hits = batch_df.count()
    
    if hits!=0: 
        start = time.time()
        #Creating the ABSOLUTETIME column
        batch_df = batch_df.withColumn("ABSOLUTETIME", 25*(col('ORBIT_CNT')*1e-9*3564+col('BX_COUNTER')*1e-9+(col('TDC_MEAS')*1e-9)/30))
        
        #Dataframe containing only informations about scintillator events
        batch_df_scint = (batch_df.filter('(FPGA==1) AND (TDC_CHANNEL == 128)')
                        .select(['ORBIT_CNT', 'ABSOLUTETIME'])
                        .groupBy('ORBIT_CNT')
                        .min()
                        .withColumnRenamed("min(ABSOLUTETIME)", 'ABSOLUTETIME_SCINT')
                        .drop('min(ORBIT_CNT)')
                       )
        #Dividing the dataframe between chambers
        
        batch_df_ch0 = batch_df.filter('(FPGA==0) AND (TDC_CHANNEL >= 0) AND (TDC_CHANNEL < 64)')
        batch_df_ch1 = batch_df.filter('(FPGA==0) AND (TDC_CHANNEL >= 64) AND (TDC_CHANNEL < 128)')
        batch_df_ch2 = batch_df.filter('(FPGA==1) AND (TDC_CHANNEL >= 0) AND (TDC_CHANNEL < 64)')
        batch_df_ch3 = batch_df.filter('(FPGA==1) AND (TDC_CHANNEL >= 64) AND (TDC_CHANNEL < 128)')
        
        batch_dfs = [batch_df_ch0, batch_df_ch1, batch_df_ch2, batch_df_ch3]
        
        # Counting hits for each chamber

        hits_ch0 = batch_df_ch0.count()
        hits_ch1 = batch_df_ch1.count()
        hits_ch2 = batch_df_ch2.count()
        hits_ch3 = batch_df_ch3.count()

        end = time.time()-start
        #Total active channels histogram
        
        hist1 = {}
        
        #Channels per orbit histogram
            
        hist2 = {}
        min_hist, max_hist = batch_df.agg(min("ORBIT_CNT"), max("ORBIT_CNT")).rdd.flatMap(lambda x: x).collect()
        binning = list(linspace(0, 4e8, 30))

        #Active channels in orbits in which the scintillator is active histogram
        hist3 = {}

        #Drifttime histogram
        hist4 = {}
        #Binning for driftime - arbitrario perché a volte non troviamo né max né min.. da rivedere.
        binning_drift = list(linspace(0,800, 40))
        
        
        
        for chamber in [0,1,2,3]:
            hist1[chamber] = {}
            hist2[chamber] = {}
#             hist3[chamber] = {}
#             hist4[chamber] = {}

            bins, counts = (
                batch_dfs[chamber].select('TDC_CHANNEL')
                .rdd.map(lambda x: x.TDC_CHANNEL)
                .histogram(list(arange((chamber % 2)*64,(chamber % 2 +1)*64,1)))
            )
            
            bins2, counts2 = (
                batch_dfs[chamber].groupBy("ORBIT_CNT","TDC_CHANNEL").count()
                .select('ORBIT_CNT')
                .rdd.map(lambda x: x.ORBIT_CNT)
                .histogram(binning)
            )
            
#             #Filtering only useful hits (avoid scintillators) and use inner join to consider coincident events
#             batch_dfs[chamber] = batch_dfs[chamber].filter('NOT ((FPGA==1) AND (TDC_CHANNEL == 128))')\
#                                                    .join(batch_df_scint, ["ORBIT_CNT"], "inner")
                
#             #Creating driftime and select only positive values
#             batch_dfs[chamber] = batch_dfs[chamber].withColumn('DRIFTIME', (col('ABSOLUTETIME')-col('ABSOLUTETIME_SCINT'))*1e9+time_offset_by_chamber[chamber])\
#                                 .where(col('DRIFTIME')>0) #Is it possible to remove it?
            
#             bins3, counts3 = (
#                 batch_dfs[chamber].select('TDC_CHANNEL')
#                 .rdd.map(lambda x: x.TDC_CHANNEL)
#                 .histogram(list(arange((chamber % 2)*64,(chamber % 2 +1)*64,1)))
#             )

#             bins4, counts4 = (
#                 batch_dfs[chamber].select('DRIFTIME')
#                 .rdd.map(lambda x: x.DRIFTIME)
#                 .histogram(binning_drift)
#             )

            hist1[chamber]['bins'] = list(map(int,bins)) #must convert to python integers
            hist1[chamber]['counts'] = list(map(int,counts))
        
            hist2[chamber]['bins'] = list(map(int,bins2))
            hist2[chamber]['counts'] = list(map(int,counts2))
            
#             hist3[chamber]['bins'] = list(map(int,bins3))
#             hist3[chamber]['counts'] = list(map(int,counts3))
        
#             hist4[chamber]['bins'] = list(map(int,bins4))
#             hist4[chamber]['counts'] = list(map(int,counts4))

        #Producing the results dictionary
        result = {
            "hits" : hits,
            "hits_per_chamber": [hits_ch0, hits_ch1, hits_ch2, hits_ch3],
            "hist_1": hist1,
            "hist_2": hist2
#             ,
#             "hist_3": hist3,
#             "hist_4": hist4
        }
        
        #Sending the json to the producer
        producer.send('topic_results', json.dumps(result).encode('utf-8'))
#         producer.flush()
        
    else: 
        pass
    
